# 학교 알리미 접속해서 동작구 내 고등학교 이름 및 인원 수를 수집하기

학교 알리미: https://www.schoolinfo.go.kr  
참고) 런타임 > 모두 실행 시 셀레니움 브라우저 창 로딩이 덜 되어서 오류가 날 수 있습니다.

In [ ]:
# 1. Colab에 필요한 패키지 설치
!pip install selenium
!pip install webdriver_manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 12.5 MB/s eta 0:00:00


In [ ]:
# 2. Chrome 브라우저 설치
!apt-get update  # 패키지 목록 업데이트
!apt-get install -y wget unzip  # wget과 unzip 패키지 설치
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -  # Google의 리눅스 서명 키 추가
!sh -c 'echo "deb [arch=amd64] https://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list'  # Chrome 브라우저를 다운로드할 수 있도록 저장소 추가
!apt-get update  # 패키지 목록 다시 업데이트
!apt-get install -y google-chrome-stable  # 최신 안정 버전의 Chrome 브라우저 설치
!apt-get install -y fonts-nanum # 한글폰트 설치

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,788 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive

drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/Colab Notebooks/fintech_edu_2025/03.데이터_수집/03_31_셀레니움_활용_데이터_수집

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/fintech_edu_2025/03.데이터_수집/03_31_셀레니움_활용_데이터_수집


In [ ]:
# Selenium과 webdriver-manager를 사용한 Chrome 브라우저 실행 코드
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # 크롬 최신 버전 동기화
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import bs4
import time

# WebDriver 설치 및 브라우저 실행 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-popup-blocking") # 팝업 차단 옵션
# options.add_argument("--lang=ko-KR") # 언어 한국어 설정
# options.add_argument("--accept-language=ko-KR") # Accept-Language 헤더 설정
options.add_argument("--disable-extensions") # 확장 프로그램 비활성화
options.add_argument("--headless")  # ★★★헤드리스 모드로 실행 (브라우저 창을 표시하지 않음) --> 리눅스 또는 colab인 경우만
options.add_argument("window-size=1920x1080") # 헤드리스 모드로 실행 (명시적으로 표기)
options.add_argument("--no-sandbox")  # 샌드박스 모드 비활성화 (가상 환경에서 안정적인 실행을 위해)
options.add_argument("--disable-dev-shm-usage")  # /dev/shm 사용 비활성화 (메모리 부족 방지)
options.add_argument("--disable-gpu")  # GPU 가속 비활성화 (필요시 사용)
options.add_argument("start-maximized")  # 브라우저 창 최대화
options.add_argument("disable-infobars")  # 정보 표시줄 비활성화
options.add_argument("--disable-extensions")  # 브라우저 확장 프로그램 비활성화

# user-agent 값 설정
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
options.add_argument(f"user-agent={user_agent}")

# WebDriver로 Chrome 브라우저 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# 웹페이지 파싱 될때까지 최대 3초 기다림
driver.implicitly_wait(3)

In [ ]:
# 학교 알리미 사이트로 이동
schoolInfo = "https://www.schoolinfo.go.kr/"
driver.get(schoolInfo)

In [ ]:
# 팝업창 열렸는지 확인하기
closeXpath = '//*[@id="pop0000000000000045"]/dl/dd/span/span/div/label/span'
closeXpathEle = driver.find_element(By.XPATH, closeXpath)
closeXpathEle.click()

In [ ]:
# 현재 화면 확인
driver.save_screenshot("./schoolInfo/schoolInfo.jpg")

print(f"현재 페이지: {driver.current_url}")

현재 페이지: https://www.schoolinfo.go.kr/Main.do


In [ ]:
# '고등학교'라는 검색어 검색창에 태워서 보내기
searchXPath = '//*[@id="SEARCH_KEYWORD"]'

schoolInfoSearchEle = driver.find_element(By.XPATH, searchXPath)

schoolInfoSearchEle.send_keys('고등학교')

In [ ]:
schoolInfoSearchEle.send_keys(Keys.ENTER)
time.sleep(2)

True

In [ ]:
# 현재 화면 확인
driver.save_screenshot("./schoolInfo/search_highschool.jpg")

True

In [ ]:
# 설립구분
selectState = '//*[@id="FNDN_SC_CD"]'

# 셀렉트 박스 요소 찾기
select_box = driver.find_element(By.XPATH, selectState)

In [ ]:
# Select 객체를 사용하여 셀렉트 박스 제어
select = Select(select_box)
select.select_by_index(0) # default

# 현재 화면 확인
driver.save_screenshot("./schoolInfo/default_state.jpg")

True

In [ ]:
# 지역구분1 " 서울특별시 선택"
selectSido = '//*[@id="SIDO_CODE"]'

# 셀렉트 박스 요소 찾기
select_box = driver.find_element(By.XPATH,selectSido)

# Select 객체를 사용하여 셀렉트 박스 제어
select = Select(select_box)

# 지역구분1 = 서울특별시
select.select_by_visible_text("서울특별시")
time.sleep(2)

In [ ]:
# 현재 화면 확인
driver.save_screenshot("./schoolInfo/Seoulsi.jpg")

True

In [ ]:
# 지역구분2 " 동작구 선택"
selectGugun = '//*[@id="GUGUN_CODE"]'

# 셀렉트 박스 요소 찾기
select_box = driver.find_element(By.XPATH,selectGugun)

# Select 객체를 사용하여 셀렉트 박스 제어
select = Select(select_box)
select.select_by_visible_text("동작구")
time.sleep(2)

In [ ]:
# 현재 화면 확인
driver.save_screenshot("./schoolInfo/dongjakgu.jpg")

True

In [ ]:
### 검색 버튼 클릭
searchBtn = '//*[@id="contents"]/div/div[1]/div/a'
searchBtnEl = driver.find_element(By.XPATH, searchBtn)
searchBtnEl.click()
time.sleep(2)

In [ ]:
# 현재 화면 확인
driver.save_screenshot("./schoolInfo/dongjakgu_search_list.jpg")

True

In [ ]:
# 더보기 버튼 클릭
try:
    addxPath = '//*[@id="contents"]/div/div[2]/a'
    driver.find_element(By.XPATH, addxPath).click()
    time.sleep(2)
except:
    print("더보기 버튼 없음")

True

In [ ]:
# 현재 화면 확인
driver.save_screenshot("./schoolInfo/dongjakgu_list.jpg")

True

In [ ]:
# 데이터 수집
schoolName = []
totalNumber = []
try:
    page_num = 2
    for page in range(0, page_num):
        html = driver.page_source
        bsObj = bs4.BeautifulSoup(html,"html.parser")
        bsObj = bsObj.find('div', attrs={"class":"search_result srDiv1"})
        divs = bsObj.findAll('div', attrs={"class":"basic_data"})

        for i in range(0, len(divs)):
            print(f"{page+(i+1)}번째 학교 수집")
            sName = divs[i].find(name="a").text
            schoolName.append(sName)

            indata = divs[i].findAll(name="div", attrs={"class":"l_box"})[1].find(name="span",attrs={"class":"md"}).text
            # 줄 단위로 나눈 후 공백 제거
            lines = indata.split()

            # 인원수 부분 찾기
            for part in lines:
                if '명' in part:
                    # '명'을 제거하여 숫자만 추출
                    total_students = part.replace('명', '')
                    # print(total_students)
                    totalNumber.append(total_students)
                    break
        # 다음 페이지 버튼 클릭
        nextBtn = '//*[@id="contents"]/div/div[3]/div[1]/a[5]'

        driver.find_element(By.XPATH, nextBtn).click()
        time.sleep(3)
except Exception as e:
    print(e)
    pass

<ipython-input-51-f2dfbc77b50f>:10: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  divs = bsObj.findAll('div', attrs={"class":"basic_data"})
<ipython-input-51-f2dfbc77b50f>:17: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  indata = divs[i].findAll(name="div", attrs={"class":"l_box"})[1].find(name="span",attrs={"class":"md"}).text


1번째 학교 수집
2번째 학교 수집
3번째 학교 수집
4번째 학교 수집
5번째 학교 수집
2번째 학교 수집
3번째 학교 수집


In [ ]:
import pandas as pd

# 데이터 가지고 DataFrame 생성
dongjakHs = pd.DataFrame( zip(schoolName, totalNumber), columns=["학교명","인원"] )

# 생성한 DataFrame 설정
dongjakHs["인원"] = dongjakHs.인원.str.replace(",","")
dongjakHs["인원"] = dongjakHs["인원"].astype(int)
dongjakHsDf = dongjakHs.sort_values(by=["인원"], ascending=False).sort_index(ascending=True)

# 잘 편집되었는지 출력
dongjakHsDf

,학교명,인원
0,경문고등학교,658
1,동작고등학교,732
2,서울공업고등학교,742
3,성남고등학교,1013
4,수도여자고등학교,820
5,숭의여자고등학교,689
6,영등포고등학교,538
